<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Modules-for-this-notebook" data-toc-modified-id="Modules-for-this-notebook-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Modules for this notebook</a></span></li></ul></li><li><span><a href="#Computing-Degree-Centrality" data-toc-modified-id="Computing-Degree-Centrality-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Computing Degree Centrality</a></span><ul class="toc-item"><li><span><a href="#Computing-it-in-netwprkx" data-toc-modified-id="Computing-it-in-netwprkx-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Computing it in <code>netwprkx</code></a></span></li></ul></li><li><span><a href="#Eigenvector-Centrality" data-toc-modified-id="Eigenvector-Centrality-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Eigenvector Centrality</a></span><ul class="toc-item"><li><span><a href="#Computing-Eigenvalues-with-eigh" data-toc-modified-id="Computing-Eigenvalues-with-eigh-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Computing Eigenvalues with <code>eigh</code></a></span></li><li><span><a href="#The-Power-Method" data-toc-modified-id="The-Power-Method-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>The Power Method</a></span></li><li><span><a href="#Computing-it-in-networkx" data-toc-modified-id="Computing-it-in-networkx-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Computing it in <code>networkx</code></a></span></li></ul></li><li><span><a href="#Closeness-Centrality" data-toc-modified-id="Closeness-Centrality-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Closeness Centrality</a></span></li><li><span><a href="#Betweenness-Centrality" data-toc-modified-id="Betweenness-Centrality-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Betweenness Centrality</a></span></li><li><span><a href="#Example:-15th-century-Florentine-marriages" data-toc-modified-id="Example:-15th-century-Florentine-marriages-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Example: 15th-century Florentine marriages</a></span><ul class="toc-item"><li><span><a href="#The-example" data-toc-modified-id="The-example-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>The example</a></span></li><li><span><a href="#Compute-centralities" data-toc-modified-id="Compute-centralities-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Compute centralities</a></span></li><li><span><a href="#Drawing-graphs-based-on-centrality" data-toc-modified-id="Drawing-graphs-based-on-centrality-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Drawing graphs based on centrality</a></span></li></ul></li><li><span><a href="#Code-corner-(not-covered-in-class-explicitly)" data-toc-modified-id="Code-corner-(not-covered-in-class-explicitly)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Code corner (not covered in class explicitly)</a></span></li></ul></div>

<h1>CS4423-Networks: Week 7 (26+27 Feb 2025) 

# Part 2: Computing Centrality Measures
Niall Madden, 
School of Mathematical and Statistical Sciences  
University of Galway


This Jupyter notebook, and PDF and HTML versions, can be found at https://www.niallmadden.ie/2425-CS4423/#Week07

<div class="rc"><font size="-1"><em>This notebook was written by Niall Madden, adapted from notebooks by Angela Carnevale.</em></div>

### Modules for this notebook

In [ ]:
import networkx as nx
import numpy as np
opts = { "with_labels": True,  "node_color": "gold"} # gold nodes today

np.set_printoptions(precision=3)    # just display arrays to 3 decimal places
np.set_printoptions(suppress=True)  # avoid scientific notation (better for matrices)

from queue import Queue # Use this in computing distances
import yaml    # for saving and diplaying data, especially dictionaries
import pandas as pd  # summarising data
import matplotlib.pyplot as plt

## Computing Degree Centrality

Computing the *Degree Centrality* of a graph is easy, and there are many ways to do it. Here we'll look at one way involving the adjancy matrix, since the core idea will be used again for Closeness Centraility

We'll start with an example: where $deg(5)=4$, $deg(4)=3$ and $deg(3)=2$.  

As an extra trick, we'll force `networkx` to order the nodes lexigraphically, by
* Creating an empty graph
* adding the nodes first (in the order I want)
* then add the edges.

In [ ]:
G1 = nx.Graph()                # empty graph
G1.add_nodes_from(range(6))    # add nodes 0,1,2,3,4,5 in that oder
EdgeList = [[5,0],[5,1],[5,2],[5,3],[4,3],[4,2],[4,1]]
G1.add_edges_from(EdgeList) 

In [ ]:
nx.draw(G1,**opts)

Compute the adjacency matrix, $A_1$ (as as numpy array), and then multiply by a vector of ones, thus computing the row sums of $A_1$. Then we normalise:

In [ ]:
A1 = nx.adjacency_matrix(G1).toarray();
n = G1.order()
e = np.ones((n,1))    # vector of ones
n = G1.order()
degree_vector = A1@e/(n-1)   # normalised 
for i in range(6):
    print(f"Node {i} has degree {degree_vector[i]}")

### Computing it in `netwprkx`

Though it is hardly needed, one can compute the degree cenrality of this network  using the `nx.degree_centrality()` method. Note that this returns a dictionary:

In [ ]:
CD = nx.degree_centrality(G1)
print(CD)
print(f"\nThe degree centrality of Node 3 is {CD[3]:.3f}")

## Eigenvector Centrality

To compute the Eigenvector Centrality of nodes in  network, $G$:
* Compute the adjacency matrix, $A$.
* Compute the largest, positive eigenvalue of $A$ (since $A$ is symmetric, this is unique)
* It has a corresponding positive eigenvector,  $\vec v$, which we can scale so that $v^v=1$.
* $v_i$ is the Eigenvector Centrality node $i$.




### Computing Eigenvalues with `eigh`

We can use `np.linalg.eig()` which computes the eigenvalues and eigenvectors of a matrix:\
`l, V = np.linalg.eig(A)`
computes 
* `l` : an array of length $n$ containing the eigenvalues of $A$. (Note: we can't call this array `lambda`, since that is a keyword in Python.
* `V`: a $n \times n$ matrix; column $i$ of $V$ is the eigenvector corresponding to the eigenvalues $\lambda_i$.

(Note: since $A$ s symmetric, it can be faster to use the `np.linalg.eigh()` function)

In [ ]:
l, V = np.linalg.eig(A1)
print(f"The eigenvalues of A are {l}")

We can see that there is an eigenvalue listed which is positive, amd larger than the rest. Let's look at the corresponding eigenvector. We make have to scale by $-1$, if the enties are negative:

In [ ]:
i_max = np.argmax(l)    # get index of largest eigenvalue
v = V[:,i_max]*np.sign(V[0,i_max])   # set v to be corresponding e'vec; ensure it is positive
print("v=",v)

In [ ]:
for i in range(6):
    print(f"Node {i} has eigenvector centrality {v[i]:7.4}")

### The Power Method

There are subfields in the *Numerical Linerar Algebra* dedicated to computing estimates for eigenvalues and eigenvectors. When we only need one eigenvalue, and it is the largest,    use the **[Power method](https://en.wikipedia.org/wiki/Power_iteration)**:

1. start with any $u = (1, 1, \dots, 1)$, say;

2. keep replacing $u \gets Au$ until $u/\|u\|$ becomes stable ...

**Questions** Does this work? Meaning:
* Does the sequence actually converge?
* Does it return the correct values?

We won't study the theory of that - but will check an example.

Here is an implementation. We'll just do 10 iterations. By rights, we should use a while loop to iterate until successive estimates are sufficiently close to each other.

In [ ]:
n = G1.order()
u = np.ones((n,1)); u=u/np.linalg.norm(u)
for i in range(10):   
    v = A1 @ u  # update u
    l = v[0]/u[0] # appriximate the eigenvalue
    u = v/np.linalg.norm(v) # normalise it

The result we get is as follows (compare yourself with the value computed earlier)

In [ ]:
print(u)

### Computing it in `networkx`

To compute eigenvector centraility in `networkx`, we can use the `nx.eigenvector_centrality` function, which returns a dictionary.

In [ ]:
CE = nx.eigenvector_centrality(G1)
print(yaml.dump(CE))  # looks better than "print(CV)"

print(f"\nThe Eigenvector  centrality of Node 3 is {CE[3]:.3f}")

## Closeness Centrality

We learned yesterday that the **normalised closeness centrality** of node $i$ is
$$
C_i^C =  \frac{n-1}{\sum_{j=1}^n d_{ij}}.
$$

To compute this, for all nodes, we could construct the *distance matrix* for the graph. For that, we need to compute the distance between every pair of nodes. As we learned last week. that can be done with *BFS*. We learned how to do that in Week 6 (Part 1). Here is a different implementation...

* The following `python` function implements
BFS for shortest distance from a previous lecture.  
* It takes a graph $G = (X, E)$ and a vertex $x \in X$
as its arguments. 
* It returns a **dictionary**, which assigns to each node its distance to $x$.

In [ ]:
def distances(G, x): 
    # 1. init: set up the dictionary and a queue
    dists = { y: None for y in G } # distances
    Q = Queue() # queue of nodes to be visited
    dists[x] = 0
    Q.put(x)
    
    # 2. loop
    while not Q.empty(): 
        y = Q.get() 
        for z in G.neighbors(y):
            if dists[z] is None:
                dists[z] = dists[y] + 1
                Q.put(z)
    
    # 3. stop here
    return dists

Let's check it works for Node 0

In [ ]:
distances(G1,0)

Next we use these values to build the *distance matrix*, $D_1$

In [ ]:
D1 = np.zeros_like(A1)
for i in range(n):
    d_i =  distances(G1,i)
    D1[i,:]=list(d_i.values())

In [ ]:
print(D1)

Now compute the *distance sum* vector, $\vec s$

In [ ]:
n=G1.order()
s = D1 @ np.ones((n,1))
print(s)

Finally, compute the Closeness Centrality vector:

In [ ]:
CC = (n-1)/s   # note: using entrywise division
print(CC)

Compare with the `networkx` function:

In [ ]:
print(nx.closeness_centrality(G1))

## Betweenness Centrality

From yesterday: the **betweenness centrality**, $c_i^B$ of node $i$  is defined as
$$
  c_i^B = \sum_{j} \sum_{k} \frac{n_i(j,k)}{n(j,k)}, \qquad  {j \neq k \neq i}
$$
where
* $n(j,k)$ denotes the _number_  of shortest paths from  node $j$ to node $k$, and 
* $n_i(j,k)$ denotes the  number of those shortest paths \emph{passing through} node $i$.


Them the **normalised betweenness centrality**, $C_i^B$ of node $i$ is
$$ C_i^B = \frac{c_i^B}{(n-1)(n-2)} $$

Before we delve into the algorithms, let's take a simple network to study:

In [ ]:
G4 = nx.Graph() 
G4.add_edges_from(['ab','ac','bd','cd','de','df']) # Example
nx.draw(G4,**opts)

The quantities, particularly, $n_i(j,k)$, can take some work to compute.
Yet again, we use a variant on **BFS**.

First for any given any node, we need to compute all its **predecessors** on the shortest paths between it and every other node. That is, if `z` is a predecessor of `x` if it is a neighbour `x`, and on the shortest path between `x` and `y`. 

This is then used to count the _number_ of shortest paths between a pair of nodes.


Our function works as follows:
1. Takes the graph G  and node x as inputs
2. Returns a dictionary, `preds` where `preds[y]` is the list of predecessors of x in the paths from y to x.

In [ ]:
def predecessors(G, x):
    """ Computes the predecssors of Node x in G"""
    # 1. init: set up the two dictionaries and queue
    dists = { y: None for y in G } # distances
    preds = { y: [] for y in G } 
    Q = Queue()
    dists[x] = 0 # 
    Q.put(x)
    
    # 2. loop
    while not Q.empty():
        y = Q.get()
        for z in G.neighbors(y):
            if dists[z] is None:
                dists[z] = dists[y] + 1
                preds[z].append(y)
                Q.put(z)
            elif dists[z] > dists[y]:
                preds[z].append(y)
    
    # 3. stop here
    return preds

Let's check it it works by computing all the predecessors of `a`:

In [ ]:
p = predecessors(G4,'a') ## check our work
print(p)

Using the **predecessor lists** with respect to $x$, the **shortest paths** from $x$ to $y$ can be enumerated recursively:
* if $y = x$: the shortest path from $x$ to itself is the empty path starting and ending at $x$.
* else, if $y \neq x$ then each shortest path from $x$ to $y$ travels through
  exactly one of $x$'s predecessors ... and ends in $y$.

In [ ]:
def shortest_paths(G, x, y):
    if x == y:
        return [[x]]
    paths = []
    pred_x_y = predecessors(G, x)[y] # predicessors of x in paths x to y
    # print(f"preds of {y} are {pred_x_y}")  # uncomment for more info
    for z in pred_x_y:
        for path in shortest_paths(G, x, z):
            paths.append(path + [y])
    return paths

Check if it works

In [ ]:
shortest_paths(G4, 'a', 'f')

Finally, we can compute the _betweenness_ of a node:

In [ ]:
def betweeness(G):
    CB = { i : 0.0 for i in G }
    n = G.order()
    for i in G: 
        for j in G:
            for k in G:
                paths_jk = shortest_paths(G, j, k)
                n_jk = len(paths_jk)
                n_i_jk = 0
                for p in paths_jk:
                    if i in p[1:-1]: # exclude enpoint
                        n_i_jk+=1
                CB[i] += n_i_jk/n_jk
        CB[i] /= ((n-1)*(n-2)) # normalise
    return(CB)

In [ ]:
betweeness(G4)

Naturally, this can also be done in `networkx`:

In [ ]:
nx.betweenness_centrality(G4)

## Example: 15th-century Florentine marriages

There is a famous network used to represent the marriage network of sixteen families in Florence, originally developed to showed how the Medici family gained power and took control of Florence by creating a high number of inter-marriages with the other families; see [Wikipedia](https://en.wikipedia.org/wiki/Strategic_network_formation)

### The example

In [ ]:
FFG = nx.florentine_families_graph()
print(f"There are {FFG.order()} nodes and {FFG.size()} links in the network.")
pos = nx.spring_layout(FFG, seed=0) # record for layer use.
nx.draw(FFG, **opts, pos=pos)

### Compute centralities 
Let's compute the centralities of each (using `networkx` methods):

In [ ]:
CD = nx.degree_centrality(FFG)
CE = nx.eigenvector_centrality(FFG)
CC = nx.closeness_centrality(FFG)
CB = nx.betweenness_centrality(FFG)

Let's display the results in a `pandas` data frameL

In [ ]:
pd.DataFrame({
    'Key': list(CD.keys()),
    'Degree': list(CD.values()),
    'Eigenv': list(CE.values()),
    'Closen': list(CC.values()),
    'Betwee': list(CB.values())
}).sort_values('Degree', ascending=False)

### Drawing graphs based on centrality
We'll finish by plotting the graphs again, but this time using the centralities measures to control the node sizes:

In [ ]:
node_sizes = [CD[node]*6000 for node in FFG.nodes()]
nx.draw(FFG, with_labels=True, node_size=node_sizes, node_color='skyblue', pos=pos)
plt.title('Degree')

In [ ]:
node_sizes = [CE[node] * 6000 for node in FFG.nodes()]
nx.draw(FFG, with_labels=True, node_size=node_sizes, node_color='plum', pos=pos)
plt.title('Eigevector')

In [ ]:
node_sizes = [CC[node]*6000 for node in FFG.nodes()]
nx.draw(FFG, with_labels=True, node_size=node_sizes, node_color='yellow', pos=pos)
plt.title('Closeness')

In [ ]:
node_sizes = [CB[node]*6000 for node in FFG.nodes()]
nx.draw(FFG, with_labels=True, node_size=node_sizes, node_color='lime', pos=pos)
plt.title('Betweenness')

In [ ]:
node_sizes = [CC[node]*6000 for node in FFG.nodes()]
node_colors = [CC[node] for node in FFG.nodes()] 
nx.draw(FFG, with_labels=True, node_size=node_sizes, node_color=node_colors, cmap=plt.cm.cool, font_size=10, pos=pos)
plt.title('Closeness (colour and size)')

## Code corner (not covered in class explicitly)

This is a list a list of functions, and coding ideas, used in this notebook.

How to make a dictionary from two lists: one of keys, one of values, using `zip`. In this case, we'll make one based on the list of nodes, and vector of degree centralities:

In [ ]:
degree_vector # Note this is a (6,1) array, not a (6,) array: need to flatten

In [ ]:
CD_dict = dict(zip(range(6), list(degree_vector.flatten() )))
print(CD_dict)

<div class="alert alert-block alert-info">Finished here <b>Thursday</b></div>